# Article: [Predicting TCR-Epitope Binding Specificity Using Deep Metric Learning and Multimodal Learning](https://www.mdpi.com/2073-4425/12/4/572)

## **Objects**

#### *1. Develop a Computational Model: The paper aims to create a convolutional neural network model that utilizes deep metric learning and multimodal learning techniques to predict interactions between T cell receptors (TCRs) and Major Histocompatibility Complex class I-peptide complexes (pMHC).*

#### *2. Simultaneous TCR-Epitope Binding Prediction: The paper seeks to perform two critical tasks in TCR-epitope binding prediction: identifying the TCRs that bind a given epitope from a TCR repertoire and identifying the binding epitope of a given TCR from a list of candidate epitopes. The goal is to achieve accurate predictions for both tasks simultaneously.*

##### *3. Gain Insights into Binding Specificity: The paper aims to provide insights into the factors that determine TCR-epitope binding specificity, including the identification of key amino acid sequence patterns and positions within the TCR that are important for binding specificity. Additionally, the paper challenges the assumption that physical proximity to epitopes is the sole determinant of TCR-epitope specificity.*

## Packages

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch

2023-11-04 11:45:58.704767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 11:45:59.143085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-04 11:45:59.143107: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-04 11:45:59.213672: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 11:46:00.329761: W tensorflow/stream_executor/platform/de

## One-hot enocoding.

In [2]:
positives = pd.read_csv("./positive.csv")
negatives = pd.read_csv("./negative.csv")

amino_acids = []

def add_to_amino_acids(a_sequence: str):
    for acid in a_sequence:
        if acid not in amino_acids:
            amino_acids.append(acid)

positives.stack().reset_index(drop=True).apply(add_to_amino_acids)

amino_acids.sort()

amino_acid_label_encoder = LabelEncoder()
amino_acid_label_encoder.fit(amino_acids)

all_amino_acids = amino_acid_label_encoder.transform(amino_acids)

def feature_map(p_sequence):
    return [tf.one_hot(amino_acid_label_encoder.transform(list(x)), len(all_amino_acids)) for x in p_sequence]

data_cd3r = feature_map(positives["cdr3"])
data_epitope = feature_map(positives["antigen.epitope"])

2023-11-04 11:46:05.124189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-04 11:46:05.124473: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-04 11:46:05.124487: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matts-computer): /proc/driver/nvidia/version does not exist
2023-11-04 11:46:05.126619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Representation

<center>
    <img src="gene.jpg" alt="Figure 1">
</center>

### 2.2. CDR3B and Epitope Sequence Representation:



**Data Representation Goals:**

  1. Convert amino acid sequences from string format to a numeric representation.
  2. Develop a numerical procedure utilizing Atchley representation to capture physical and biochemical properties.
  3. Create matrices with specified dimensions through padding to accommodate varying sequence lengths.

### Sentence Construction with the Atchley Representation in both CDR3B and the Epitope
Fixed: Instead of constructing the sentences manually, I constructed the sentences utilizing the BertTokenizer in relateion to the amino_aciv_vocab.txt.

### Procedure encoding CDR3B and Eptiope Amino Acid Sequences as Numerical Matrices

In [3]:
def convert_to_space_separated_string(series):
    return ' '.join(series)

tokenizer = BertTokenizer(vocab_file="./amino_acid_vocab.txt")

def construct_sentences(dataframe):
    cdr3_sentences = dataframe["cdr3"]
    epitope_sentences = dataframe["antigen.epitope"]
    return cdr3_sentences, epitope_sentences

def pad_sentences(sentences, max_length):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.stack(input_ids), torch.stack(attention_masks)

max_length = 32

positives = pd.read_csv("./positive.csv")

for column in positives.columns:
    positives[column] = positives[column].apply(convert_to_space_separated_string)

cdr3_sentences, epitope_sentences = construct_sentences(positives)

cdr3_input_ids, cdr3_attention_masks = pad_sentences(cdr3_sentences, max_length)
epitope_input_ids, epitope_attention_masks = pad_sentences(epitope_sentences, max_length)

cdr3_combined = torch.cat((cdr3_input_ids, cdr3_attention_masks), dim=1)
epitope_combined = torch.cat((epitope_input_ids, epitope_attention_masks), dim=1)

cdr3_train_data, cdr3_test_data = train_test_split(cdr3_combined, test_size=0.2, random_state=42)
epitope_train_data, epitope_test_data = train_test_split(epitope_combined, test_size=0.2, random_state=42)
print(cdr3_input_ids)
print(epitope_input_ids)

tensor([[[2, 5, 6,  ..., 0, 0, 0]],

        [[2, 5, 6,  ..., 0, 0, 0]],

        [[2, 5, 6,  ..., 0, 0, 0]],

        ...,

        [[2, 5, 6,  ..., 0, 0, 0]],

        [[2, 5, 6,  ..., 0, 0, 0]],

        [[2, 5, 6,  ..., 0, 0, 0]]])
tensor([[[ 2,  8, 10,  ...,  0,  0,  0]],

        [[ 2,  8, 10,  ...,  0,  0,  0]],

        [[ 2,  8, 10,  ...,  0,  0,  0]],

        ...,

        [[ 2, 10, 10,  ...,  0,  0,  0]],

        [[ 2, 10, 10,  ...,  0,  0,  0]],

        [[ 2, 10, 10,  ...,  0,  0,  0]]])


### Initialization of Training 

In [5]:
from transformers import *

config = BertConfig.from_json_file("./bert_config.json")

model = BertForMaskedLM(config=config) # change this line and flag what we're using

device = "cuda" if torch.cuda.is_available() else "cpu"

cdr3_train_data = cdr3_train_data.to(device)
cdr3_test_data = cdr3_test_data.to(device)
epitope_train_data = epitope_train_data.to(device)
epitope_test_data = epitope_test_data.to(device)

model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25, 32, padding_idx=0)
      (position_embeddings): Embedding(128, 32)
      (token_type_embeddings): Embedding(2, 32)
      (LayerNorm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=32, out_features=32, bias=True)
              (key): Linear(in_features=32, out_features=32, bias=True)
              (value): Linear(in_features=32, out_features=32, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=32, out_features=32, bias=True)
              (LayerNorm): LayerNorm((32,), eps=1e-12, elementwise_affine=True)
      

### Triplet Loss Function:
Article: [PyTorch Metric Learning](https://kevinmusgrave.github.io/pytorch-metric-learning/#:~:text=This%20customized%20triplet%20loss%20has,than%200.3%20will%20be%20discarded.)

In [7]:
!pip install pytorch_metric_learning 
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.reducers import ThresholdReducer
from pytorch_metric_learning.regularizers import LpRegularizer
from pytorch_metric_learning import losses

In [8]:
loss_func = losses.TripletMarginLoss(distance = CosineSimilarity(), reducer = ThresholdReducer(high=0.3), embedding_regularizer = LpRegularizer())

#### Analyze the embedding between the CDR3 and the Epitope.

#### 1. BertForPreTraining


In [10]:
import torch
from transformers import BertTokenizer, BertForPreTraining, BertForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the BERT tokenizer
config = BertConfig.from_json_file("./bert_config.json")

model = BertForMaskedLM(config=config) # change this line and flag what we're using

# Define your training data
# Here, we'll use dummy data for simplicity. In practice, you'd use a large text corpus.
train_data = "Hello, my name is [MASK]. I live in [MASK]."

# Tokenize and convert the training data into input tensors
input_ids = tokenizer.encode(train_data, add_special_tokens=True, return_tensors="pt")
labels = input_ids.clone()

# Create a dataset and data collator
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path="amino_acid_vocab.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bert-pretraining",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=10_000,
)

# Create a Trainer for pretraining
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Start pretraining
trainer.train()
trainer.train()
trainer.train()
trainer.train()
trainer.train()

# Save the pretrained model

/home/matthandzel/anaconda3/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Creating features from dataset file at amino_acid_vocab.txt
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/matthandzel/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.A

  0%|          | 0/2 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running training *****
  Num examples = 25
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2
  Number of trainable parameters = 19097


{'train_runtime': 0.0791, 'train_samples_per_second': 316.107, 'train_steps_per_second': 25.289, 'train_loss': 3.226839065551758, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

...

{'train_runtime': 0.0764, 'train_samples_per_second': 327.324, 'train_steps_per_second': 26.186, 'train_loss': 3.397287130355835, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 0.0771, 'train_samples_per_second': 324.172, 'train_steps_per_second': 25.934, 'train_loss': 1.5246171951293945, 'epoch': 1.0}


TrainOutput(global_step=2, training_loss=1.5246171951293945, metrics={'train_runtime': 0.0771, 'train_samples_per_second': 324.172, 'train_steps_per_second': 25.934, 'train_loss': 1.5246171951293945, 'epoch': 1.0})

#### 2. BertForMaskedLM


#### 3. BertForNextPrediction